In [4]:
# using python 3.6
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import itertools
from moviepy.editor import VideoFileClip
%matplotlib inline

## CAMERA CALIBRATION 

In [5]:
# camera calibration and distrotion coef
# chess board is 9x6 (internal corners)

# the object points (stay same) and image point lists
rows = 6
cols = 9

objpts = list(zip(list(range(cols))*rows, 
             list(itertools.chain(*[[i]*cols for i in range(rows)])), 
             [0]*rows*cols))
# convert from tuple to list to arr 
objpts = np.array([list(i) for i in objpts], np.float32) 
objptsLst = []
imgptsLst = []

In [6]:
# perform calibration by iterating through calib. images
image_files = os.listdir('./camera_cal/')

for image_file in image_files:
    image_file = './camera_cal/'+image_file # get image path
    image = plt.imread(image_file)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # find the image points (chess board corners in that image)
    ret, imgpts = cv2.findChessboardCorners(gray, (cols, rows), None)
    if ret: # if a valid value has been returned
        imgptsLst.append(imgpts)
        objptsLst.append(objpts)

In [7]:
# use img and obj points to find calibration mat and distortion coef
ret, mtx, dis, rvecs, tvecs = cv2.calibrateCamera(objptsLst, imgptsLst,
                                                 gray.shape[::-1], None, None)

## IMAGE PROCESSING FUNCTIONS

Please note that only some of the functions below are actually used in the final image processing pipeline.

In [8]:
# undistorts any input image by same camera as above
def undistort_img(img, mtx, dis):
    return cv2.undistort(img, mtx, dis, None, mtx)

In [9]:
# remove noise from img using gaussian blurring
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [10]:
# sharpens the edges of image
def sharpen(img):
    kernel = np.array([[-1,-1,-1,-1,-1],
                         [-1,2,2,2,-1],
                         [-1,2,16,2,-1],
                         [-1,2,2,2,-1],
                         [-1,-1,-1,-1,-1]]) / 16.
    sharp_img = cv2.filter2D(img, -1, kernel)
    return sharp_img

In [11]:
# apply sobel (gradient) thresh
def apply_sobel_thresh(img, sobel_kernel=15, gradx_thresh=[0,150], angle_thresh=[0.5,2], show=False):
    # convert to grayscale
    if len(img.shape) > 2:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else: 
        gray = img
    # apply sobel operator
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # get abs of sobel
    sobel_x = np.absolute(sobel_x)
    sobel_y = np.absolute(sobel_y)
    # get the angel
    grad_angle = np.arctan2(sobel_y, sobel_x)
    # normalize sobel_x and convert to 8-bit (0 to 255)
    max_p = np.max(sobel_x)
    sobel_x = np.uint8(255*sobel_x/max_p)
    # generate binary image based on thresholds
    low_x, high_x = gradx_thresh
    low_ang, high_ang = angle_thresh
    binary_grad = np.multiply(np.multiply(sobel_x>=low_x, sobel_x<=high_x), 
                np.multiply(grad_angle>=low_ang, grad_angle<=high_ang))
    if show:
        plt.imshow(binary_grad, cmap='gray')
        plt.show()
    
    return np.uint8(binary_grad)

In [12]:
# apply Hue and Saturation thresh
# must insert a color img
def apply_HS_thresh(img, S_thresh=[100,255], H_thresh=[20,75], show=False):
    
    # convert to hls image
    hls_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # get the channels
    H,L,S = hls_img[:,:,0], hls_img[:,:,1], hls_img[:,:,2]
    # apply HLS and COLOR thresholds
    low_H, high_H = H_thresh
    low_S, high_S = S_thresh
    binary_hls = np.multiply(np.multiply(H>=low_H, H<=high_H),
                    np.multiply(S>=low_S, S<=high_S))
    if show:
        plt.imshow(binary_hls, cmap='gray')
        plt.show()
    return np.uint8(binary_hls)

In [13]:
# apply RGB thresh
# must insert a color RGB img
def apply_RGB_thresh(img, R_thresh=[190,255], G_thresh=[190,255], B_thresh=[190,255], show=False):
    
    # get the different channels
    R,G,B = img[:,:,0], img[:,:,1], img[:,:,2]
    # apply COLOR thresholds
    low_R, high_R = R_thresh
    low_G, high_G = G_thresh
    low_B, high_B = B_thresh
    binary_rgb = np.multiply(np.multiply(np.multiply(R>=low_R, R<=high_R),
                    np.multiply(G>=low_G, G<=high_G)),np.multiply(B>=low_B, B<=high_B))
    if show:
        plt.imshow(binary_rgb, cmap='gray')
        plt.show()
    return np.uint8(binary_rgb)

In [14]:
# add threshold images (binary)
def add_binary_images(img1, img2):
    return np.uint8((img1+img2)>0)

In [15]:
# mask to remove unnecessary points
def mask(bit_img):
    yy, xx = bit_img.shape[:2]
    mask_coords = np.int32([[500,450],[780,450],[1150,yy],[200,yy]])
    mask = np.zeros_like(bit_img)
    mask = cv2.fillConvexPoly(mask, mask_coords, 255) # we only have one color channel
    img_masked = cv2.bitwise_and(bit_img, mask)
    return img_masked

In [16]:
# perform perspective transform on image
def warp(img):
    # the source and destination points
    # get img dims 
    h,w = img.shape[:2]
    # the source and destination points
    src = np.float32([[575,470],[750,470],[260,670],[1100,670]])
    dst = np.float32([[200,0],[w-200,0],[200,h],[w-200,h]])
    #src = np.float32([[595,450],[690,450],[1040,660],[260,660]])
    #dst = np.float32([[300,100],[1060,100],[1060,660],[250,660]])
    # get the transformation matrix
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src) # get the inverse transform
    warped = cv2.warpPerspective(img, M, 
            (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
    return warped, M_inv

In [17]:
# sliding windows to detect lanes
def sliding_windows(bit_warped, nwinds=9, return_curvature=True):
    
    # histogram
    hist = np.sum(bit_warped[bit_warped.shape[0]//2:,:], axis=0)
    #plt.plot(hist)
    out_img = np.dstack((bit_warped, bit_warped, bit_warped))*255
    
    # find the midpoint of x-axis
    midpoint = np.int(hist.shape[0]/2)
    
    # find start of left and right windows
    leftx_base = np.argmax(hist[:midpoint])
    rightx_base = np.argmax(hist[midpoint:]) + midpoint
    window_h = np.int(bit_warped.shape[0]/nwinds)
    nonzero_pos = bit_warped.nonzero() 
    nonzero_y, nonzero_x = nonzero_pos # all the chosen pix (regarless of window)
    
    # set starting positions
    leftx_curr = leftx_base
    rightx_curr = rightx_base
    # width of the windows
    width = 100 # NOTE: var
    # threshold to recenter window
    pix_thresh = 50 # NOTE: var
    # lists to stor inds
    left_lane_idx = []
    right_lane_idx = []
    
    for window in range(nwinds):
        # starting all the way at the bottom
        y_up = bit_warped.shape[0] - (window+1)*window_h
        y_down = bit_warped.shape[0] - window*window_h
        x_left_l = leftx_curr - width
        x_left_r = leftx_curr + width
        x_right_l = rightx_curr - width
        x_right_r = rightx_curr + width
        # draw the windows (first left and then right)
        cv2.rectangle(out_img, (x_left_l, y_up), (x_left_r, y_down), (0,255,0), 2)
        cv2.rectangle(out_img, (x_right_l, y_up), (x_right_r, y_down), (0,255,0), 2)
        # find pixels that are within this window - the 'good' pixels
        # the [0] at the end is because the original list is just 1-D arr NOT 2D
        good_left = ((nonzero_y < y_down) & (nonzero_y >= y_up) &\
                    (nonzero_x >= x_left_l) & (nonzero_x <= x_left_r)).nonzero()[0]
        good_right = ((nonzero_y < y_down) & (nonzero_y >= y_up) &\
                    (nonzero_x >= x_right_l) & (nonzero_x <= x_right_r)).nonzero()[0]
        # append them to list
        left_lane_idx.append(good_left)
        right_lane_idx.append(good_right)
        # recenter window (for next iter) if needed
        if len(good_left) > pix_thresh:
            leftx_curr = np.int(np.mean(nonzero_x[good_left]))
        if len(good_right) > pix_thresh:
            rightx_curr = np.int(np.mean(nonzero_x[good_right]))
        
    # concatenate the arrays
    left_lane_idx = np.concatenate(left_lane_idx)
    right_lane_idx = np.concatenate(right_lane_idx)
    # get list of x and y coords
    left_x = nonzero_x[left_lane_idx]
    left_y = nonzero_y[left_lane_idx]
    right_x = nonzero_x[right_lane_idx]
    right_y = nonzero_y[right_lane_idx]
    # fit a polynomial
    left_poly_fit = np.polyfit(left_y, left_x, 2)
    right_poly_fit = np.polyfit(right_y, right_x, 2)
    
    # Generate x and y values of the lanes
    ploty = np.linspace(0, bit_warped.shape[0]-1, bit_warped.shape[0] )
    left_fitx = left_poly_fit[0]*ploty**2 + \
                left_poly_fit[1]*ploty + left_poly_fit[2]
    right_fitx = right_poly_fit[0]*ploty**2 +\
                right_poly_fit[1]*ploty + right_poly_fit[2]
    
    return_list = [ploty, left_fitx, right_fitx, left_poly_fit, right_poly_fit]
    
    if return_curvature:
        y_ = np.max(ploty)
        curvature_left = ((1 + (2*left_poly_fit[0]*y_ + 
                    left_poly_fit[1])**2)**1.5) / np.absolute(2*left_poly_fit[0])
        curvature_right = ((1 + (2*right_poly_fit[0]*y_ + 
                    right_poly_fit[1])**2)**1.5) / np.absolute(2*right_poly_fit[0])
        return_list.append(curvature_left)
        return_list.append(curvature_right)
        
    return return_list

In [18]:
# uses previous lane lines to create new lane lines
# img is a binary, transformed image
def new_lane_lines(img, left_fit, right_fit, margin=100, return_curvature=True):
    
    # get non-zero points
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # make sure points are within margin
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    y_ = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*y_**2 + left_fit[1]*y_ + left_fit[2]
    right_fitx = right_fit[0]*y_**2 + right_fit[1]*y_ + right_fit[2]
    
    return_list = [y_, left_fitx, right_fitx, left_fit, right_fit]
    
    if return_curvature:
        y_ = np.max(y_)
        curvature_left = ((1 + (2*left_fit[0]*y_ + 
                    left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
        curvature_right = ((1 + (2*right_fit[0]*y_ + 
                    right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
        return_list.append(curvature_left)
        return_list.append(curvature_right)
        
    return return_list

In [19]:
# draw lane lines to the original image
def draw_lane_lines(orig_img, y_, left_fitx, right_fitx, M_inv):
    
    h,w = orig_img.shape[:2]
    lines_img = np.zeros([h,w]).astype(np.uint8)
    color_lines_img = np.dstack([lines_img, lines_img, lines_img])
    
    # gen points (x,y)
    left_pts = np.array([np.transpose(np.vstack([left_fitx, y_]))])
    # right lane points in reverse to work with cv2.fillPoly
    right_pts = np.array([np.flipud(np.transpose(np.vstack([right_fitx, y_])))])    
    all_pts = np.hstack([left_pts, right_pts])
    # first add the lines
    cv2.polylines(color_lines_img, np.int32(left_pts), isClosed=False, color=[255,0,0], thickness=50)
    cv2.polylines(color_lines_img, np.int32(right_pts), isClosed=False, color=[0,0,255], thickness=50)
    # now fill the area between the two lanes in green
    cv2.fillPoly(color_lines_img, np.int_([all_pts]), (0,255,0))
    # unwarp the image
    warped_lines = cv2.warpPerspective(color_lines_img, M_inv, (w,h))
    # add this to the original image
    output_img = cv2.addWeighted(orig_img, 1, warped_lines, 0.3, 0)
    
    return output_img   

In [20]:
# add the curvature of each lane to the image
def add_curvature_info(img, left_cur, right_cur):
    out = np.copy(img)
    cv2.putText(out, 'left curvature: {0:.2f} meters'.format(left_cur), 
                (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.putText(out, 'right curvature: {0:.2f} meters'.format(right_cur),
                (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    return out

## Image Pipeline Function
Uses appropriate functions from above to create a pipeline that takes in unmarked road image and returns an image annotated with the lane lines.

In [21]:
# takes in original image, performs various functions to generate an output
# image with the lane lines detected

bad_img_store = []
left_fit = [] # blank lists used to test if first image
right_fit = []

def pipeline(orig_img, mtx=mtx, dis=dis):
    
    global left_fit, right_fit
    
    # plot image for checking
    #plt.imshow(orig_img)
    plt.show()
    
    try:
        # undistort image
        undist_img = undistort_img(orig_img, mtx, dis)
        # sharpen image
        sharpen_img = sharpen(undist_img)
        # warp the image
        bin_warped_img, M_inv = warp(undist_img)
        # apply hls and rgb thresholds
        rgb_thresh_img = apply_RGB_thresh(bin_warped_img)
        hls_thresh_img = apply_HS_thresh(bin_warped_img)
        bin_img = add_binary_images(rgb_thresh_img, hls_thresh_img)
        # detect lanes using sliding windows if first image
        if len(left_fit)==0 or len(right_fit)==0:
            y_, left, right, left_fit, right_fit, cur_left, cur_right =\
            sliding_windows(bin_img, nwinds=9, return_curvature=True)
            #print('up')
        else: # for rest of the images simple extrapolate using existing lines
            y_, left, right, left_fit, right_fit, cur_left, cur_right =\
            new_lane_lines(bin_img, left_fit, right_fit, margin=100, return_curvature=True)
            #print('down')
            if np.sum(left == right) == len(left):
                #print('equal')
                y_, left, right, left_fit, right_fit, cur_left, cur_right =\
                sliding_windows(bin_img, nwinds=9, return_curvature=True)
                #print('reverted to starting calc again')

        # draw the lanes in original image
        color_img_lines = draw_lane_lines(orig_img, y_, left, right, M_inv)
        # add curvature info
        output_img = add_curvature_info(color_img_lines, cur_left, cur_right)
        return output_img
    
    except Exception as e: # capture any errors
        print(e)
        bad_img_store.append(orig_img)
        return orig_img
    

## Annotating the Video

In [26]:
#### APPLY PIPELINE TO VIDEO ####
output_video_file = './results/lane_lines_video.mp4'
input_video = VideoFileClip('./project_video.mp4')
output_video = input_video.fl_image(pipeline)
%time output_video.write_videofile(output_video_file, audio=False) 

[MoviePy] >>>> Building video ./results/lane_lines_video.mp4
[MoviePy] Writing video ./results/lane_lines_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [01:19<00:00, 16.15it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./results/lane_lines_video.mp4 

Wall time: 1min 20s
